<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Model Training and Deployment

In [1]:
import pandas as pd
import pandas_profiling
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score,confusion_matrix, f1_score
from tabpy_tools.client import Client

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 700)

In [2]:
app_data = pd.read_csv("app_data_informative.csv")

In [3]:
#Encode the Rating_Score into a categorical variable so that we can build a classification model on it 
#(less than 2,2-4,4-4.5,above 4.5)
app_data.loc[app_data['RATING_SCORE'] <= 4.5,'RATING_SCORE_CODED'] = 'BAD'
app_data.loc[(app_data['RATING_SCORE'] > 4.5) & (app_data['RATING_SCORE'] <= 5),'RATING_SCORE_CODED'] = 'GOOD'

#Encoding the categorical features so that they can be used as model input
le_cat = preprocessing.LabelEncoder()
app_data['CATEGORY'] = le_cat.fit_transform(app_data['CATEGORY'])

le_con_rat = preprocessing.LabelEncoder()
app_data['CONTENT_RATING'] = le_con_rat.fit_transform(app_data['CONTENT_RATING'])

le_ec = preprocessing.LabelEncoder()
app_data['EDITORS_CHOICE'] = le_ec.fit_transform(app_data['EDITORS_CHOICE'])

le_free = preprocessing.LabelEncoder()
app_data['FREE'] = le_free.fit_transform(app_data['FREE'])

le_iap = preprocessing.LabelEncoder()
app_data['IAP'] = le_iap.fit_transform(app_data['IAP'])

"""
cat = pd.get_dummies(app_data['CATEGORY'], prefix='CATEGORY_',drop_first=True)
con_rat = pd.get_dummies(app_data['CONTENT_RATING'], prefix='CONTENT_',drop_first=True)
app_data['EDITORS_CHOICE'] = pd.get_dummies(app_data['EDITORS_CHOICE'])
app_data['FREE'] = pd.get_dummies(app_data['FREE'])
app_data['IAP'] = pd.get_dummies(app_data['IAP'])
"""


"\ncat = pd.get_dummies(app_data['CATEGORY'], prefix='CATEGORY_',drop_first=True)\ncon_rat = pd.get_dummies(app_data['CONTENT_RATING'], prefix='CONTENT_',drop_first=True)\napp_data['EDITORS_CHOICE'] = pd.get_dummies(app_data['EDITORS_CHOICE'])\napp_data['FREE'] = pd.get_dummies(app_data['FREE'])\napp_data['IAP'] = pd.get_dummies(app_data['IAP'])\n"

In [4]:
app_data.corr()['RATING_SCORE']

Unnamed: 0                  0.008474
CATEGORY                    0.013902
CONTENT_RATING              0.054038
EDITORS_CHOICE              0.075222
FREE                       -0.072216
IAP                         0.142646
INSTALLS                    0.102282
PRICE                       0.026392
REQUIRED_ANDROID_VERSION   -0.059023
NUMBER_REVIEWS              0.107812
RATING_SCORE                1.000000
SIZE                        0.063009
Name: RATING_SCORE, dtype: float64

In [5]:
app_data = app_data.drop(['Unnamed: 0','RATING_SCORE','DEVELOPER','TITLE','LAST_UPDATED'], axis = 1)

# app_data = pd.concat([app_data,cat, con_rat],axis=1)
#app_data = app_data.drop(['Unnamed: 0','RATING_SCORE','CATEGORY','CONTENT_RATING','DEVELOPER','TITLE','LAST_UPDATED'], axis = 1)

In [6]:
app_data.columns

Index(['CATEGORY', 'CONTENT_RATING', 'EDITORS_CHOICE', 'FREE', 'IAP', 'INSTALLS', 'PRICE', 'REQUIRED_ANDROID_VERSION', 'NUMBER_REVIEWS', 'SIZE', 'RATING_SCORE_CODED'], dtype='object')

In [7]:
# pandas_profiling.ProfileReport(app_data)

In [8]:
#Split into two variables, One for explanotory variable and other for the explained variable
X = app_data.drop('RATING_SCORE_CODED',axis=1)
y = app_data['RATING_SCORE_CODED']

#Split into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state=12)

In [9]:
#Normalize/ Scale the explanatory variable values
from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\admin\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  


In [10]:
#LogisticRegression
lr=LogisticRegression(random_state=123)
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
accuracy_score(y_test, y_pred)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7669976598570616

In [11]:
#Decision Tree
dt=DecisionTreeClassifier(random_state=123)
dt.fit(X_train,y_train)
y_pred=dt.predict(X_test)
cm=confusion_matrix(y_test,y_pred)
accuracy_score(y_test, y_pred)

0.7008095629624944

In [12]:
print(lr.coef_)

[[ 0.03939434 -0.03836649  0.03618084 -0.29252112  0.13488642 -0.34493983
   0.02336343  0.09346886  0.29871209  0.01190262]]


In [13]:
def app_rating_predictor( _arg1, _arg2, _arg3, _arg4, _arg5, _arg6, _arg7, _arg8, _arg9, _arg10):
    import pandas as pd
    
    #Get the new app's data in a dictionary
    row = {'CATEGORY': _arg1,
           'CONTENT_RATING': _arg2,
           'EDITORS_CHOICE': _arg3,
           'FREE': _arg4,
           'IAP': _arg5, 
           'INSTALLS': _arg6,
           'PRICE': _arg7, 
           'REQUIRED_ANDROID_VERSION': _arg8,
           'NUMBER_REVIEWS': _arg9,
           'SIZE': _arg10}
    
    #Convert it into a dataframe
    new_app_data = pd.DataFrame(data = row,index=[0])
    
    #Apply the label encodings
    new_app_data['CATEGORY'] = le_cat.fit_transform(new_app_data['CATEGORY'])
    new_app_data['CONTENT_RATING'] = le_con_rat.fit_transform(new_app_data['CONTENT_RATING'])
    new_app_data['EDITORS_CHOICE'] = le_ec.fit_transform(new_app_data['EDITORS_CHOICE'])
    new_app_data['FREE'] = le_free.fit_transform(new_app_data['FREE'])
    new_app_data['IAP'] = le_iap.fit_transform(new_app_data['IAP'])
    
    new_app_data = sc.transform(new_app_data)
    pred_rating = lr.predict_proba(new_app_data)
    return [rating[1] for rating in pred_rating]

In [14]:
app_rating_predictor('COMMUNICATION','Mature 17+',False,False,True,1000,0.0,0.0,423111,4.1)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.


array([[0.49112104, 0.50887896]])

run this in conda prompt 
1. cd C:\Users\admin\Anaconda3\Lib\site-packages\TabPy-0.6.1
2. startup.cmd


In [17]:
client = Client('http://localhost:9004/')

In [18]:
client.deploy('app_rating_predictor', app_rating_predictor,'Test', override = True)